# Documentation

This analysis tool reads .igc files from an indicated directory, filters the data using the `DataAnalyzer` helper class and then analyzes the speed data by applying the `CAnalyzer` algorithm.

Inputs:
- INPUT_DIRECTORY: path to the directory containing .igc files
- OUTPUT_DIRECTORY: output data will be exported to this directory
- FILE_EXTENSION: set to `.igc`

# Imports

In [ ]:
import os
import sys
import pandas as pd
from datetime import datetime
from typing import List, Tuple

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import algorithms.speed_analyzer as speeds
import helpers.file_processor as file_processor
import helpers.data_visualizer as data_visualizer
import algorithms.c_values_analyzer as c_values_analyzer

# Variables

In [ ]:
INPUT_DIRECTORY: str = "INPUT_DIRECTORY/" # end with "/
OUTPUT_DIRECTORY: str = "OUTPUT_DIRECTORY/" # end with /
FILE_EXTENSION: str = ".igc"

# Initialisation

In [ ]:
file_processor = file_processor.FileProcessor()
speed_analyzer = speeds.SpeedAnalyzer()
data_visualizer = data_visualizer.DataVisualizer()
c_analyzer = c_values_analyzer.CAnalyzer()

theoretical_reference: pd.DataFrame = pd.read_csv(f"{constants.THEORETICAL_REFERENCE_PATH}")
original_reference: pd.DataFrame = pd.read_csv(f"{constants.ORIGINAL_REFERENCE_PATH}")

timestamp: str = datetime.now().strftime("%Y%m%d-%H%M%S")

# File Listing

In [ ]:
file_paths: List[str] = file_processor.get_file_paths(path=INPUT_DIRECTORY, file_extension=FILE_EXTENSION)

print(f"Importing files for the following conditions:")
print(f"--> Directory: {INPUT_DIRECTORY}")
print(f"--> File extension: {FILE_EXTENSION}")
print(f"--> Theoretical polar: {constants.THEORETICAL_REFERENCE_PATH.split("/")[-1]}")
print(f"--> Original reference: {constants.ORIGINAL_REFERENCE_PATH.split("/")[-1]}")
print()
print("Files:")
print(f"--> Found {len(file_paths)} files.")
print(f"--> The processing is initiated.")

# File Processing

In [ ]:
data_raw: pd.DataFrame = speed_analyzer.process_raw_data(file_paths=file_paths)

# Data Processing

In [ ]:
# igc data
theoretical_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=theoretical_reference, reference=True)
original_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=original_reference, reference=True)
data_raw_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=data_raw)

print(f"Filtered data:")
print(f"--> Data points for filtered theoretical reference: {len(theoretical_reference_filtered)} (lost {len(theoretical_reference) - len(theoretical_reference_filtered)})")
print(f"--> Data points for filtered original reference: {len(original_reference_filtered)} (lost {len(original_reference) - len(original_reference_filtered)})")
print(f"--> Data points for filtered tracklogs: {len(data_raw_filtered)} (lost {len(data_raw) - len(data_raw_filtered)})")
print()

# smooth and filter igc data
smoothed_data_raw: pd.DataFrame = speed_analyzer.savgol_filter(data=data_raw_filtered)
smoothed_data_grouped: pd.DataFrame = speed_analyzer.group_data(data=smoothed_data_raw)

print(f"Data smoothing:")
print(f"--> During smoothing, the raw data points were reduced from {len(data_raw_filtered)} to {len(smoothed_data_raw)} (lost {len(data_raw_filtered) - len(smoothed_data_raw)}).")
print(f"--> During grouping, the smoothed data points were reduced from {len(smoothed_data_raw)} to {len(smoothed_data_grouped)} (lost {len(smoothed_data_raw) - len(smoothed_data_grouped)}).")
print()

# add airspeed to datasets
print(f"Manipulating datasets:")
print(f"--> The datasets are being extended:")
print(f"----> The original reference dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"----> The theoretical reference dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"----> The experimental dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"--> Vertical speeds are converted to be positive:")
print(f"----> The vertical speeds of the original reference dataset are being converted to be positive.")
print(f"----> The vertical speeds of the theoretical reference dataset are being converted to be positive.")
print(f"----> The vertical speeds of the experimental dataset are being converted to be positive.")

airspeed_original_reference: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=original_reference_filtered))
airspeed_theoretical_reference: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=theoretical_reference_filtered))
airspeed_smoothed_data_grouped: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=smoothed_data_grouped))

print(f"--> The datasets have been extended and manipulated.")
print()

# calculate c values for datasets (simplified and optimized algorithm)
print(f"C values:")
print(f"--> Calculating the c values for the following conditions:")
print(f"----> Altitude: {constants.ALTITUDE} m")
print(f"----> Air density: {constants.AIR_DENSITY} kg/m^3")
print(f"----> Gravity: {constants.GRAVITY} m/s^2")
print(f"----> Wing area: {constants.WING_AREA} m^2")
print(f"----> Takeoff mass: {constants.MASS} kg")
print(f"--> Calculating the c values for the following datasets:")
print(f"----> The c values are being calculated for the original reference dataset (simplified & optimized algorithm).")
print(f"----> The c values are being calculated for the theoretical reference dataset (simplified & optimized algorithm).")
print(f"----> The c values are being calculated for the experimental dataset (simplified & optimized algorithm).")

c_values_original_reference_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_original_reference)
c_values_original_reference_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_original_reference, algorithm=True)
c_values_theoretical_reference_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_theoretical_reference)
c_values_theoretical_reference_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_theoretical_reference, algorithm=True)
c_values_experimental_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_smoothed_data_grouped)
c_values_experimental_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_smoothed_data_grouped, algorithm=True)

print(f"--> The c values have been calculated.")
print()

print(f"Export data to CSV:")
print(f"--> The original reference dataset is being exported to the output directory (simplified algorithm): {timestamp}_SJf_c-values-original-reference_simplified_nicolas-huber.csv")
print(f"--> The original reference dataset is being exported to the output directory (optimized algorithm): {timestamp}_SJf_c-values-original-reference_optimized_nicolas-huber.csv")
print(f"--> The theoretical reference dataset is being exported to the output directory (simplified algorithm): {timestamp}_SJf_c-values-theoretical-reference_simplified_nicolas-huber.csv")
print(f"--> The theoretical reference dataset is being exported to the output directory (optimized algorithm): {timestamp}_SJf_c-values-theoretical-reference_optimized_nicolas-huber.csv")
print(f"--> The experimental dataset is being exported to the output directory (simplified algorithm): {timestamp}_SJf_c-values-experimental_simplified_nicolas-huber.csv")
print(f"--> The experimental dataset is being exported to the output directory (optimized algorithm): {timestamp}_SJf_c-values-experimental_optimized_nicolas-huber.csv")

speed_analyzer.export_to_csv(data=c_values_original_reference_simplified, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_c-values-original-reference_simplified_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_original_reference_optimized, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_c-values-original-reference_optimized_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_theoretical_reference_simplified, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_c-values-theoretical-reference_simplified_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_theoretical_reference_optimized, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_c-values-theoretical-reference_optimized_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_experimental_simplified, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_c-values-experimental_simplified_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_experimental_optimized, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_c-values-experimental_optimized_nicolas-huber.csv")

# Visualize Theoretical Reference

In [ ]:
data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title="Theoretische Ca-Werte (vereinfachter Algorithmus)")
data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title="Theoretische Cw-Werte (vereinfachter Algorithmus)")
data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title="Theoretische Ca-Werte (optimierter Algorithmus)")
data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title="Theoretische Cw-Werte (optimierter Algorithmus)")


# Visualize Results of Simplified Algorithm

In [ ]:
data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (vereinfachter Algorithmus, originale Daten)')
data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (vereinfachter Algorithmus, originale Daten)')
deviation_ca_original_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Abweichung der Ca-Werte (vereinfachter Algorithmus, originale Daten)')
deviation_cw_original_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Abweichung der Cw-Werte (vereinfachter Algorithmus, originale Daten)')

data_visualizer.visualize_c_values(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (vereinfachter Algorithmus, optimierte Daten)')
data_visualizer.visualize_c_values(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (vereinfachter Algorithmus, optimierte Daten)')
deviation_ca_experimental_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Abweichung der Ca-Werte (vereinfachter Algorithmus, optimierte Daten)')
deviation_cw_experimental_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Abweichung der Cw-Werte (vereinfachter Algorithmus, optimierte Daten)')

# Visualize Results of Optimized Algorithm

In [ ]:
data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (optimierter Algorithmus, originale Daten)')
data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (optimierter Algorithmus, originale Daten)')
deviation_ca_original_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Abweichung der Ca-Werte (optimierter Algorithmus, originale Daten)')
deviation_cw_original_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Abweichung der Cw-Werte (optimierter Algorithmus, originale Daten)')

data_visualizer.visualize_c_values(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (optimierter Algorithmus, optimierte Daten)')
data_visualizer.visualize_c_values(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (optimierter Algorithmus, optimierte Daten)')
deviation_ca_experimental_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Abweichung der Ca-Werte (optimierter Algorithmus, optimierte Daten)')
deviation_cw_experimental_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Abweichung der Cw-Werte (optimierter Algorithmus, optimierte Daten)')

# Report

In [ ]:
score_original_ca_simplified: float = c_analyzer.score_stats(stats=deviation_ca_original_simplified)
score_original_cw_simplified: float = c_analyzer.score_stats(stats=deviation_cw_original_simplified)
score_experimental_ca_simplified: float = c_analyzer.score_stats(stats=deviation_ca_experimental_simplified)
score_experimental_cw_simplified: float = c_analyzer.score_stats(stats=deviation_cw_experimental_simplified)
score_original_ca_optimized: float = c_analyzer.score_stats(stats=deviation_ca_original_optimized)
score_original_cw_optimized: float = c_analyzer.score_stats(stats=deviation_cw_original_optimized)
score_experimental_ca_optimized: float = c_analyzer.score_stats(stats=deviation_ca_experimental_optimized)
score_experimental_cw_optimized: float = c_analyzer.score_stats(stats=deviation_cw_experimental_optimized)

c_analyzer.print_report(score_original_ca_simplified=score_original_ca_simplified, score_original_cw_simplified=score_original_cw_simplified, score_experimental_ca_simplified=score_experimental_ca_simplified, score_experimental_cw_simplified=score_experimental_cw_simplified, score_original_ca_optimized=score_original_ca_optimized, score_original_cw_optimized=score_original_cw_optimized, score_experimental_ca_optimized=score_experimental_ca_optimized, score_experimental_cw_optimized=score_experimental_cw_optimized)

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")